In [ ]:
import io
from pathlib import Path
from tempfile import TemporaryDirectory
import zipfile

import pandas
import requests

In [ ]:
year = '2023'

uri = f'https://downloads.cms.gov/Files/hcris/HOSP10FY{year}.zip'

report_dtype = {
       'RPT_REC_NUM': 'Int64',
       'PRVDR_CTRL_TYPE_CD':   'category',
       'PRVDR_NUM':            'string',
       'NPI':                  'Int64',
       'RPT_STUS_CD':          'category',
       'FY_BGN_DT':            'object',
       'FY_END_DT':            'object',
       'PROC_DT':              'object',
       'INITL_RPT_SW':         'category',
       'LAST_RPT_SW':          'category',
       'TRNSMTL_NUM':          'category',
       'FI_NUM':               'string',
       'ADR_VNDR_CD':          'string',
       'FI_CREAT_DT':          'object',
       'UTIL_CD':              'category',
       'NPR_DT':               'object',
       'SPEC_IND':             'string',
       'FI_RCPT_DT':           'object'}
report_parse_dates = [k for k,v in report_dtype.items() if v == 'object']

values_dtype = {
       'RPT_REC_NUM':          'int64',
       'WKSHT_CD':             'category',
       'LINE_NUM':             'category',
       'CLMN_NUM':             'category',
       'VALUE':                'string'
}

lookup_dtype = {
       'Variable Name':          'string',
       'Cost Report Worksheet Element': 'string',
       'WKSHT_CD':             'category',
       'LINE_NUM':             'category',
       'CLMN_NUM':             'category',
}

In [ ]:
with (
    requests.get(uri) as result,
    zipfile.ZipFile(io.BytesIO(result.content)) as zip_handler,
    TemporaryDirectory() as temp_context
):
    data_path = Path(temp_context)
    zip_handler.extractall(data_path)
    # rpt =pandas.read_csv(data_path / 'HOSP10_2019_RPT.CSV', header=0, names=report_dtype.keys(), dtype=report_dtype, parse_dates=report_parse_dates)
    values = pandas.read_csv(data_path / f'HOSP10_{year}_NMRC.CSV', header=0, names=values_dtype.keys(), dtype=values_dtype, )
    values = pandas.concat([values, pandas.read_csv(data_path / f'HOSP10_{year}_ALPHA.CSV', header=0, names=values_dtype.keys(), dtype=values_dtype, ) ], ignore_index=True)

In [ ]:
lookup = pandas.read_csv(Path.cwd() / 'etc' / 'lookup.csv', header=0, names=lookup_dtype.keys(), dtype=lookup_dtype,)

merged = values.merge(lookup, how='inner', on=['WKSHT_CD','LINE_NUM','CLMN_NUM'], )

In [ ]:
pivoted = merged.pivot(index='RPT_REC_NUM',columns='Variable Name', values='VALUE').reset_index().rename_axis(None, axis=1)

# order the columns
ordered = pivoted[['RPT_REC_NUM'] + list(lookup['Variable Name'])]

In [ ]:
ordered[ordered['State Code'] == 'NC'].to_csv(Path.cwd() / 'etc' / f'costreport_{year}.csv')